In [ ]:
ipython = get_ipython()
colab   = 'google.colab' in str(ipython)
print(ipython)

if colab:
    from google.colab import drive
    drive.mount('/content/gdrive')
    %cd '/content/gdrive/My Drive/Colab Notebooks'

!ls "./"

def MODULE_FROM_NOTEBOOK(target, source, *keys): 
    from json import load
    with open(source) as notebook:
        data = load(notebook)
        with open(target,'w') as module:
            for cell in data['cells']:
                lines = cell['source']
                if type(lines) == str:
                    lines = lines.split('\n')
                line0  = (lines or [''])[0].strip()
                haskey = lambda key: line0 == f'### {key} ###'
                if cell['cell_type'] == 'code' and any(map(haskey,keys)):
                    code = '\n'.join(lines)
                    module.write(code)
                    module.write('\n')
                     
MODULE_FROM_NOTEBOOK('./ttt_tactics.py',
                     './ttt-tactics.ipynb',
                     'HEAD','GAME','DATA','MODEL','SEARCH')

from ttt_tactics import *

!cat /proc/meminfo | grep Mem

ipynb = 'ttt-tactics-test'

%load_ext tensorboard


In [ ]:
%tensorboard --logdir "./tensorboard/"

In [ ]:
%%monitor_cell "ttt-tactics test-play"

model = AlphaZeroModel()
models = sortedfiles(f'{DIR}/model-*.h5')
model.load(models[-1])

pi = AlphaZeroPolicy(model, times=100, cp=0, tau=0)
ratios12 = testgames({1:pi,2:randompi}, iters=100)
ratios21 = testgames({2:pi,1:randompi}, iters=100)

NOTIFY(f'{str(model)} tested, {ratios12} vs {ratios21}', title=ipynb, priority=1)






In [ ]:
from rle.np_impl import dense_to_rle

@autograph.experimental.do_not_convert
def loadsamples(files, buffer_size=1024, batch_size=64, seed=None):
    proto = { 'state'  : FixedLenFeature([size_s], int64),
              'probs'  : FixedLenFeature([num_a],  float),
              'reward' : FixedLenFeature([1],      int64) }
    def loadsample(sample):
        sample = io.parse_single_example(sample,proto)
        input  = {'state'  : sample['state']}
        output = {'value'  : sample['reward'], 
                  'policy' : sample['probs']}
        return input,output 
    dataset = data.TFRecordDataset(files)            
    dataset = dataset.map(loadsample, num_parallel_calls=AUTOTUNE)
    return dataset

path    = f'{DIR}/data-1601999616.0.tfrec'       
files   = sortedfiles(path)
dataset = loadsamples(files)
actor   = 0
i       = 0
for input,output in dataset:
    # print(input['state'].shape,output['policy'].shape,output['value'].shape)      
    state = input['state']
    p     = player(state)
    if p != actor or turn(state) == 0:
        actor = p
        key   = digits(state)
        S,s,t = splitgame(state)
        print(f'{i:0>4d}: player={p}, turn={t:0>2d}')
        pprint(key)
        pprint(S)
        pprint(s)
        # pprint(state)
        print('\n')
        if i > 100:
            break
        i += 1


